In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pickle
import datetime

In [3]:
MMM = pd.read_csv('MMM.csv')

MMM.head()

MMM.tail()

AdjClose = MMM['Adj Close'].values

MMMRtn = np.divide(AdjClose[1:] - AdjClose[:-1], AdjClose[:-1])

JPM = pd.read_csv('JPM.csv')
JPM_Close = JPM['Adj Close'].values
JPM_Rtn = np.divide(JPM_Close[1:] - JPM_Close[:-1], JPM_Close[:-1])

MSFT = pd.read_csv('MSFT.csv')
MSFT_Close = MSFT['Adj Close'].values
MSFT_Rtn = np.divide(MSFT_Close[1:] - MSFT_Close[:-1], MSFT_Close[:-1])

PLD = pd.read_csv('PLD.csv')
PLD_Close = PLD['Adj Close'].values
PLD_Rtn = np.divide(PLD_Close[1:] - PLD_Close[:-1], PLD_Close[:-1])

AMZN = pd.read_csv('AMZN.csv')
AMZN_Close = AMZN['Adj Close'].values
AMZN_Rtn = np.divide(AMZN_Close[1:] - AMZN_Close[:-1], AMZN_Close[:-1])

XOM = pd.read_csv('XOM.csv')
XOM_Close = XOM['Adj Close'].values
XOM_Rtn = np.divide(XOM_Close[1:] - XOM_Close[:-1], XOM_Close[:-1])

JNJ = pd.read_csv('JNJ.csv')
JNJ_Close = JNJ['Adj Close'].values
JNJ_Rtn = np.divide(JNJ_Close[1:] - JNJ_Close[:-1], JNJ_Close[:-1])

ATT = pd.read_csv('T.csv')
ATT_Close = ATT['Adj Close'].values
ATT_Rtn = np.divide(ATT_Close[1:] - ATT_Close[:-1], ATT_Close[:-1])


Returns = pd.DataFrame({'Date': MMM['Date'][:-2], 'MMM': MMMRtn[:-1], 'MSFT': MSFT_Rtn[:-1], 
                        'JPM': JPM_Rtn[:-1], 'PLD': PLD_Rtn[:-1],
                        'AMZN': AMZN_Rtn[:-1], 'XOM': XOM_Rtn[:-1], 
                        'JNJ': JNJ_Rtn[:-1], 'ATT': ATT_Rtn[:-1]})

Returns

,Date,MMM,MSFT,JPM,PLD,AMZN,XOM,JNJ,ATT
0,2019-07-15,0.007937,0.034548,0.023604,0.016968,-0.010929,-0.002400,0.003223,0.041476
1,2019-07-22,-0.019715,-0.031414,-0.028308,-0.019486,-0.061661,-0.040904,0.002601,0.000586
2,2019-07-29,-0.041513,0.005917,-0.028248,0.016396,-0.008589,-0.012683,0.007400,0.010828
3,2019-08-05,-0.012051,-0.011473,-0.018407,0.006844,-0.008304,-0.035855,-0.005150,0.012450
4,2019-08-12,-0.026154,-0.016865,-0.015782,-0.006069,-0.023960,0.000427,-0.027634,-0.004290
...,...,...,...,...,...,...,...,...,...
256,2024-06-10,0.014767,0.016291,0.013004,-0.025737,0.029511,0.015122,0.022056,0.043084
257,2024-06-17,-0.001953,-0.006292,0.030362,0.035452,0.022054,0.039364,-0.017412,0.038587
258,2024-06-24,-0.008514,0.046113,0.012509,0.019143,0.034929,-0.015202,0.002189,-0.017792
259,2024-07-01,0.026846,-0.029964,0.006278,0.049712,-0.027550,-0.000882,0.023211,0.002131


## Find dates when out-of-sample test is done

In [4]:
n_runs = 20
n_stock = 5
seq_len = 4
n_iter = 2000
batch = 10

outRtn_dates = []

for scene in range(n_runs):
    scene = scene*10
    out_of_sample_idx = scene + batch + seq_len
    outRtn_dates.append(Returns.iloc[out_of_sample_idx]['Date'])

In [5]:
outRtn_dates

['2019-10-21',
 '2019-12-30',
 '2020-03-09',
 '2020-05-18',
 '2020-07-27',
 '2020-10-05',
 '2020-12-14',
 '2021-02-22',
 '2021-05-03',
 '2021-07-12',
 '2021-09-20',
 '2021-11-29',
 '2022-02-07',
 '2022-04-18',
 '2022-06-27',
 '2022-09-05',
 '2022-11-14',
 '2023-01-23',
 '2023-04-03',
 '2023-06-12']

## T-bill annual rates 

In [6]:
all_rates = pd.DataFrame()

for year in np.arange(2019, 2025, 1):
#     print(year)
    rate_file = './{}-daily-treasury-rates.csv'.format(year)
    rates = pd.read_csv(rate_file)
    all_rates = pd.concat([all_rates, rates])

In [11]:
datetime.datetime.strptime("12/21/2008", "%m/%d/%Y").strftime("%Y-%m-%d")

'2008-12-21'

In [8]:
df_len = len(Returns)
for idx in range(df_len):
    date = Returns.iloc[idx]['Date']
    date = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%m/%d/%Y')
    rate = all_rates[all_rates['Date'] == date]['1 Mo'].values
#     print(rate)
    if len(rate) != 1:
        Returns.loc[idx, '1Month'] = Returns.loc[idx-1, '1Month']
    else:
        Returns.loc[idx, '1Month'] = rate

In [9]:
Returns

,Date,MMM,MSFT,JPM,PLD,AMZN,XOM,JNJ,ATT,1Month
0,2019-07-15,0.007937,0.034548,0.023604,0.016968,-0.010929,-0.002400,0.003223,0.041476,2.17
1,2019-07-22,-0.019715,-0.031414,-0.028308,-0.019486,-0.061661,-0.040904,0.002601,0.000586,2.13
2,2019-07-29,-0.041513,0.005917,-0.028248,0.016396,-0.008589,-0.012683,0.007400,0.010828,2.12
3,2019-08-05,-0.012051,-0.011473,-0.018407,0.006844,-0.008304,-0.035855,-0.005150,0.012450,2.07
4,2019-08-12,-0.026154,-0.016865,-0.015782,-0.006069,-0.023960,0.000427,-0.027634,-0.004290,2.09
...,...,...,...,...,...,...,...,...,...,...
256,2024-06-10,0.014767,0.016291,0.013004,-0.025737,0.029511,0.015122,0.022056,0.043084,5.47
257,2024-06-17,-0.001953,-0.006292,0.030362,0.035452,0.022054,0.039364,-0.017412,0.038587,5.45
258,2024-06-24,-0.008514,0.046113,0.012509,0.019143,0.034929,-0.015202,0.002189,-0.017792,5.42
259,2024-07-01,0.026846,-0.029964,0.006278,0.049712,-0.027550,-0.000882,0.023211,0.002131,5.48


In [13]:
MMM.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-15,144.657196,145.476593,143.988297,144.322739,117.084564,2822919
1,2019-07-22,144.489960,156.956528,142.617050,145.468231,118.013878,23053498
2,2019-07-29,145.342804,148.846161,141.981598,142.600327,115.687241,14875728
3,2019-08-05,141.262543,141.321075,132.140472,136.680603,110.884766,20600981
4,2019-08-12,136.304352,140.367889,130.125412,135.033447,109.548462,19983965


In [10]:
Returns.to_csv('Returns.csv', header=True, index=False)

In [12]:
out_rates = Returns[Returns['Date'].isin(outRtn_dates)]['1Month'].values
out_rates

array([1.76, 1.51, 0.57, 0.1 , 0.1 , 0.09, 0.07, 0.03, 0.02, 0.05, 0.06,
       0.07, 0.03, 0.38, 1.16, 2.45, 3.72, 4.69, 4.7 , 5.24])